In [33]:
import os
import pandas as pd
import numpy as np
import json
import sys

In [34]:
# Get the current working directory
CURRENT_DIRECTORY = os.getcwd()
# Get the parent directory
PARENT_DIRECTORY = os.path.dirname(CURRENT_DIRECTORY)
PARENT_DIRECTORY = os.path.dirname(PARENT_DIRECTORY)

sys.path.append(PARENT_DIRECTORY + '//config/')
from config import count_sequences_above_threshold, count_sequences_below_threshold, normalize_sensor_data

# Open the config file and load its content into a dictionary
config_file = open(PARENT_DIRECTORY + '\\config\\config.json')
CONFIG_DATA = json.load(config_file)

# Close the file after loading the data
config_file.close()

In [35]:
NORMALIZE_SENSOR_DATA = CONFIG_DATA['NORMALIZE_SENSOR_DATA']
WINDOW_SIZE = CONFIG_DATA['WINDOW_SIZE']
NUMBER_OF_FEATURES = 8

ROTATION_THRESHOLD = CONFIG_DATA['LAR_L_SIDESTEPS_ROLL_ROTATION_THRESHOLD']  if NORMALIZE_SENSOR_DATA == False else normalize_sensor_data( CONFIG_DATA['LAR_L_SIDESTEPS_ROLL_ROTATION_THRESHOLD'] )

print(ROTATION_THRESHOLD)

CLASSIFICATIONS = CONFIG_DATA['CLASSES_MOTIONSPEED']

LABEL_COLUMN = 'Class_MotionSpeed'
# processed data folder path
DATA_FOLDER = PARENT_DIRECTORY + '\\processed-training-data\\4-PROCESSED-DATA\TRAIN2\\'

# save numpy data folder
NUMPY_DATA_FOLDER_FILE_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\'

# v0 data (not normalized, use to generate more data)
ALL_X_TRAIN_CLASSIFY_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\training\\ALL-X-TRAIN-CLASSIFY-MOTIONSPEED-LAR-SIDESTEPS-V0.npy'
ALL_Y_TRAIN_CLASSIFY_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\training\\ALL-Y-TRAIN-CLASSIFY-MOTIONSPEED-LAR-SIDESTEPS-V0.npy'

-5


In [36]:
LABEL_TO_CATEGORY = {label: category for category, label in enumerate(CLASSIFICATIONS)}

print(LABEL_TO_CATEGORY)

{'SLOW': 0, 'AVERAGE': 1, 'FAST': 2}


In [37]:
file_names = [file for file in os.listdir(DATA_FOLDER) if file.endswith('.xlsx')and "LAR" in file and "SIDESTEPS" in file and os.path.isfile(os.path.join(DATA_FOLDER, file))]

print(file_names)

['PROC-TRAIN2-SIDESTEPS-L-LAR-110BPM.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-120BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-160BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-164BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-220BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-30BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-40BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-41BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-55BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-60BPM.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-80BPM.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-82BPM.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-LAR-110BPM.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-LAR-120BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-LAR-160BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-LAR-164BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-LAR-220BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-LAR-30BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-LAR-40BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-LAR-41BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-LAR-

In [38]:
def df2Xy(df, windowSize=5):
  X = []
  y = []

  # NORMALIZE THE SENSOR DATA
  #df["L_Pitch_Delta"] = df["L_Pitch_Delta"].apply(normalize_sensor_data)
  #df["L_Roll_Delta"] = df["L_Roll_Delta"].apply(normalize_sensor_data)
  #df["R_Pitch_Delta"] = df["R_Pitch_Delta"].apply(normalize_sensor_data)
  #df["R_Roll_Delta"] = df["R_Roll_Delta"].apply(normalize_sensor_data)

  for i in range( len(df) - windowSize + 1):
    # inputs: X rows
    # form a new input which has size of our windowSize
    input_data_list_deltas = []
    input_data_list_readings = []

    # loop through each row in our windowsize
    for j in range(windowSize):
        # fetch sensor data for this row
        row_values_deltas = df.loc[i + j, ['L_Pitch_Delta', 'L_Roll_Delta', 'R_Pitch_Delta', 'R_Roll_Delta']].values.tolist()
        row_values_readings = df.loc[i + j, ['L_Pitch', 'L_Roll', 'R_Pitch', 'R_Roll']].values.tolist()

        # add row values to the input
        input_data_list_deltas.append(row_values_deltas)
        input_data_list_readings.append(row_values_readings)

    # turnn list into array to do arthimetic
    raw_sensor_data = np.array(input_data_list_readings)
    delta_sensor_data = np.array(input_data_list_deltas)

    # remove all negative sinces we want to the total change (we dont care which direction)
    input_calculate_deltas = np.abs(delta_sensor_data)

    # calculate the total change for each sennsor value (TOTAL POSITIVE SENSOR CHANGES)
    input_calculate_deltas = np.sum(input_calculate_deltas, axis=0)

    # calculate total sequences
    input_total_sequences = count_sequences_below_threshold(raw_sensor_data, ROTATION_THRESHOLD)

    input_data_array = np.concatenate((input_calculate_deltas, input_total_sequences))

    if(np.isnan(input_data_array).any() == False):
      # turn back to list
      input_data_list = input_data_array.tolist()

      # add our input to our total inputs, marked as X
      X.append(input_data_list)

      # outputs: y labels
      label = df.loc[i + (windowSize - 1), [LABEL_COLUMN]].values.tolist()

      y.append(label)

  return (np.array(X), np.array(y))

In [39]:
ALL_X_TRAIN = np.empty((0, NUMBER_OF_FEATURES))  # List to store all X training data
ALL_Y_TRAIN = np.empty((0))  # List to store all Y training data

In [40]:
def processData(ALL_X, ALL_Y):
    for fileName in file_names:
        # Read the Excel file
        df = pd.read_excel(DATA_FOLDER + fileName)

        xTrain, yTrain = df2Xy(df, WINDOW_SIZE)

        yTrain = yTrain.reshape(-1)
        
        ALL_X = np.concatenate([ALL_X, xTrain])
        ALL_Y = np.concatenate([ALL_Y, yTrain])
        
        print('xTrain.shape:', xTrain.shape)
        print('ALL_X_TRAIN.shape:', ALL_X.shape, ' ALL_Y_TRAIN.shape:', ALL_Y.shape)
        print('-------------------------------')

    # Create the folder if it doesn't exist
    if not os.path.exists(NUMPY_DATA_FOLDER_FILE_PATH):
        os.makedirs(NUMPY_DATA_FOLDER_FILE_PATH)

    # convert to numerical labels (originally text labels)
    numerical_label = np.vectorize(LABEL_TO_CATEGORY.get)(ALL_Y)

    # Assuming your numpy array is called 'data_array'
    np.save(ALL_X_TRAIN_CLASSIFY_PATH, ALL_X)
    np.save(ALL_Y_TRAIN_CLASSIFY_PATH, numerical_label)

In [41]:
processData(ALL_X_TRAIN, ALL_Y_TRAIN)

ALL_X_TRAIN_CLASSIFY = np.load(ALL_X_TRAIN_CLASSIFY_PATH)
ALL_Y_TRAIN_CLASSIFY = np.load(ALL_Y_TRAIN_CLASSIFY_PATH)

print(ALL_X_TRAIN_CLASSIFY.shape)
print(ALL_Y_TRAIN_CLASSIFY.shape)

xTrain.shape: (2285, 8)
ALL_X_TRAIN.shape: (2285, 8)  ALL_Y_TRAIN.shape: (2285,)
-------------------------------
xTrain.shape: (4077, 8)
ALL_X_TRAIN.shape: (6362, 8)  ALL_Y_TRAIN.shape: (6362,)
-------------------------------
xTrain.shape: (3794, 8)
ALL_X_TRAIN.shape: (10156, 8)  ALL_Y_TRAIN.shape: (10156,)
-------------------------------
xTrain.shape: (3134, 8)
ALL_X_TRAIN.shape: (13290, 8)  ALL_Y_TRAIN.shape: (13290,)
-------------------------------
xTrain.shape: (2284, 8)
ALL_X_TRAIN.shape: (15574, 8)  ALL_Y_TRAIN.shape: (15574,)
-------------------------------
xTrain.shape: (8174, 8)
ALL_X_TRAIN.shape: (23748, 8)  ALL_Y_TRAIN.shape: (23748,)
-------------------------------
xTrain.shape: (7608, 8)
ALL_X_TRAIN.shape: (31356, 8)  ALL_Y_TRAIN.shape: (31356,)
-------------------------------
xTrain.shape: (6288, 8)
ALL_X_TRAIN.shape: (37644, 8)  ALL_Y_TRAIN.shape: (37644,)
-------------------------------
xTrain.shape: (4587, 8)
ALL_X_TRAIN.shape: (42231, 8)  ALL_Y_TRAIN.shape: (42231,)
-

In [ ]:
print(np.max(ALL_X_TRAIN_CLASSIFY))

103.0
